In [441]:
import pymongo
from datetime import datetime


# Output functions
def print_items_short(items: list):
    for i in items:
        print(i)

def print_items_pretty(items: list, very_pretty=False):
    print(',\n'.join(dict_to_str(i, very_pretty) for i in items))

def dict_to_str(item, recursive=False):
    strings = []

    for key in item:
        v = item[key]
        if recursive:
            if type(v) == dict:
                v = dict_to_str(v, recursive=True).replace('\n', '\n\t')
            elif type(v) == list and len(v) > 0 and type(v[0]) == dict:
                v = list_to_str(v, recursive=True).replace('\n', '\n\t')
        strings.append(f'\t{key}: {v}')
    
    return '{\n%s\n}' % ',\n'.join(strings)

def list_to_str(item: list, recursive=False):
    strings = []

    for v in item:
        if recursive:
            if type(v) == dict:
                v = dict_to_str(v, recursive=True).replace('\n', '\n\t')
            elif type(v) == list:
                v = list_to_str(v, recursive=True).replace('\n', '\n\t')
        strings.append(f'\t{v}')
    
    return '[\n%s\n]' % ',\n'.join(strings)

In [442]:
# Reset DB
client = pymongo.MongoClient('mongodb://localhost:27017/')
client.drop_database('lab4')
db = client['lab4']

# Товари (items)

1) Створіть декілька товарів з різним набором властивостей Phone/TV/Smart Watch/...

In [443]:
items = db['items']

new_phones = [
    {
        "category": "Phone",
        "model": "iPhone 16 Pro",
        "producer": "Apple",
        "price": 61499,
        "display": 6.3,
        "camera": [48, 48, 12],
        "os": "iOS"
    },
    {
        "category": "Phone",
        "model": "iPhone 13",
        "producer": "Apple",
        "price": 25499,
        "display": 6.1,
        "camera": [12, 12],
        "os": "iOS"
    },
    {
        "category": "Phone",
        "model": "Galaxy S25 Ultra",
        "producer": "Samsung",
        "price": 67999,
        "display": 6.9,
        "camera": [200, 50, 50, 10],
        "os": "Android"
    },
    {
        "category": "Phone",
        "model": "Galaxy A16",
        "producer": "Samsung",
        "price": 9499,
        "display": 6.7,
        "camera": [50, 5, 2],
        "os": "Android"
    }
]

new_smart_watches = [
    {
        "category": "Smart Watch",
        "model": "Watch SE",
        "producer": "Apple",
        "price": 12599,
        "display": 1.5,
        "wrist": [130, 180],
        "os": "WatchOS"
    },
    {
        "category": "Smart Watch",
        "model": "Galaxy Watch Ultra",
        "producer": "Samsung",
        "price": 28999,
        "display": 1.5,
        "wrist": [160, 210],
        "os": "Android"
    },
    {
        "category": "Smart Watch",
        "model": "Redmi Watch 5",
        "producer": "Xiaomi",
        "price": 4499,
        "display": 2.07,
        "wrist": [135, 205],
        "os": "Android"
    }
]

new_tvs = [
    {
        "category": "TV",
        "model": "UE43T5300AUXUA",
        "producer": "Samsung",
        "price": 15199,
        "display": 43,
        "os": "Tizen"
    },
    {
        "category": "TV",
        "model": "43PUS8009/12",
        "producer": "Philips",
        "price": 20999,
        "display": 43,
        "os": "Titan OS"
    },
    {
        "category": "TV",
        "model": "50UR78006LK",
        "producer": "LG",
        "price": 21499,
        "display": 50,
        "os": "WebOS"
    }
]

items.insert_many(new_phones)
items.insert_many(new_smart_watches)
items.insert_many(new_tvs)

InsertManyResult([ObjectId('67f55a80c9c92197da10881e'), ObjectId('67f55a80c9c92197da10881f'), ObjectId('67f55a80c9c92197da108820')], acknowledged=True)

2. Напишіть запит, який виводить усі товари (відображення у JSON)

In [444]:
print_items_short(items.find())

{'_id': ObjectId('67f55a80c9c92197da108817'), 'category': 'Phone', 'model': 'iPhone 16 Pro', 'producer': 'Apple', 'price': 61499, 'display': 6.3, 'camera': [48, 48, 12], 'os': 'iOS'}
{'_id': ObjectId('67f55a80c9c92197da108818'), 'category': 'Phone', 'model': 'iPhone 13', 'producer': 'Apple', 'price': 25499, 'display': 6.1, 'camera': [12, 12], 'os': 'iOS'}
{'_id': ObjectId('67f55a80c9c92197da108819'), 'category': 'Phone', 'model': 'Galaxy S25 Ultra', 'producer': 'Samsung', 'price': 67999, 'display': 6.9, 'camera': [200, 50, 50, 10], 'os': 'Android'}
{'_id': ObjectId('67f55a80c9c92197da10881a'), 'category': 'Phone', 'model': 'Galaxy A16', 'producer': 'Samsung', 'price': 9499, 'display': 6.7, 'camera': [50, 5, 2], 'os': 'Android'}
{'_id': ObjectId('67f55a80c9c92197da10881b'), 'category': 'Smart Watch', 'model': 'Watch SE', 'producer': 'Apple', 'price': 12599, 'display': 1.5, 'wrist': [130, 180], 'os': 'WatchOS'}
{'_id': ObjectId('67f55a80c9c92197da10881c'), 'category': 'Smart Watch', 'mod

3. Підрахуйте, скільки товарів у певної категорії

In [445]:
items.count_documents({'category': 'TV'})

3

4. Підрахуйте, скільки є різних категорій товарів

In [446]:
res = items.aggregate([
    {'$group': {'_id': '$category'}},
    {'$count': 'categories_count'}
])

print_items_short(res)

{'categories_count': 3}


5. Виведіть список всіх виробників товарів без повторів

In [447]:
items.distinct('producer')

['Apple', 'LG', 'Philips', 'Samsung', 'Xiaomi']

6. Напишіть запити, які вибирають товари за різними критеріям і їх сукупності: 
- категорія та ціна (в проміжку) - конструкція $and, 
- модель чи одна чи інша - конструкція $or,
- виробники з переліку - конструкція $in

In [448]:
res = items.find({
    '$and': [
        {'category': 'Phone'},
        {'price': {'$gt': 20000, '$lt': 62000}}
    ]
})

print_items_pretty(res)

{
	_id: 67f55a80c9c92197da108817,
	category: Phone,
	model: iPhone 16 Pro,
	producer: Apple,
	price: 61499,
	display: 6.3,
	camera: [48, 48, 12],
	os: iOS
},
{
	_id: 67f55a80c9c92197da108818,
	category: Phone,
	model: iPhone 13,
	producer: Apple,
	price: 25499,
	display: 6.1,
	camera: [12, 12],
	os: iOS
}


In [449]:
res = items.find({
    '$or': [
        {'model': 'Galaxy Watch Ultra'},
        {'model': 'Redmi Watch 5'}
    ]
})

print_items_pretty(res)

{
	_id: 67f55a80c9c92197da10881c,
	category: Smart Watch,
	model: Galaxy Watch Ultra,
	producer: Samsung,
	price: 28999,
	display: 1.5,
	wrist: [160, 210],
	os: Android
},
{
	_id: 67f55a80c9c92197da10881d,
	category: Smart Watch,
	model: Redmi Watch 5,
	producer: Xiaomi,
	price: 4499,
	display: 2.07,
	wrist: [135, 205],
	os: Android
}


In [450]:
res = items.find({'producer': {'$in': ['Philips', 'LG']}})

print_items_pretty(res)

{
	_id: 67f55a80c9c92197da10881f,
	category: TV,
	model: 43PUS8009/12,
	producer: Philips,
	price: 20999,
	display: 43,
	os: Titan OS
},
{
	_id: 67f55a80c9c92197da108820,
	category: TV,
	model: 50UR78006LK,
	producer: LG,
	price: 21499,
	display: 50,
	os: WebOS
}


7. Оновіть певні товари, змінивши існуючі значення і додайте нові властивості (характеристики) усім товарам за певним критерієм

In [451]:
# Convert inches to mm

print('before:')
print_items_short(
    items.find({'category': 'Smart Watch'}, {'_id': 0, 'model': 1, 'display': 1})
)

items.update_many(
    {'category': 'Smart Watch'},
    [{
        '$set': {
            'display': {
                '$cond': {
                    'if': {'$eq': ['$display', 1.5]},
                    'then': 40,
                    'else': 50
                }
            }
        }
    }]
)

print('\nafter:')
print_items_short(
    items.find({'category': 'Smart Watch'}, {'_id': 0, 'model': 1, 'display': 1})
)

before:
{'model': 'Watch SE', 'display': 1.5}
{'model': 'Galaxy Watch Ultra', 'display': 1.5}
{'model': 'Redmi Watch 5', 'display': 2.07}

after:
{'model': 'Watch SE', 'display': 40}
{'model': 'Galaxy Watch Ultra', 'display': 40}
{'model': 'Redmi Watch 5', 'display': 50}


In [452]:
# Add a refresh rate field

print('before:')
print_items_short(
    items.find({'category': 'TV'}, {'_id': 0, 'model': 1, 'refresh_rate': 1})
)

items.update_many({'category': 'TV'}, {'$set': {'refresh_rate': 60}})

print('\nafter:')
print_items_short(
    items.find({'category': 'TV'}, {'_id': 0, 'model': 1, 'refresh_rate': 1})
)

before:
{'model': 'UE43T5300AUXUA'}
{'model': '43PUS8009/12'}
{'model': '50UR78006LK'}

after:
{'model': 'UE43T5300AUXUA', 'refresh_rate': 60}
{'model': '43PUS8009/12', 'refresh_rate': 60}
{'model': '50UR78006LK', 'refresh_rate': 60}


8. Знайдіть товари у яких є (присутнє поле) певні властивості

In [453]:
res = items.find({'camera': {'$exists': True}})

print_items_short(res)

{'_id': ObjectId('67f55a80c9c92197da108817'), 'category': 'Phone', 'model': 'iPhone 16 Pro', 'producer': 'Apple', 'price': 61499, 'display': 6.3, 'camera': [48, 48, 12], 'os': 'iOS'}
{'_id': ObjectId('67f55a80c9c92197da108818'), 'category': 'Phone', 'model': 'iPhone 13', 'producer': 'Apple', 'price': 25499, 'display': 6.1, 'camera': [12, 12], 'os': 'iOS'}
{'_id': ObjectId('67f55a80c9c92197da108819'), 'category': 'Phone', 'model': 'Galaxy S25 Ultra', 'producer': 'Samsung', 'price': 67999, 'display': 6.9, 'camera': [200, 50, 50, 10], 'os': 'Android'}
{'_id': ObjectId('67f55a80c9c92197da10881a'), 'category': 'Phone', 'model': 'Galaxy A16', 'producer': 'Samsung', 'price': 9499, 'display': 6.7, 'camera': [50, 5, 2], 'os': 'Android'}


9. Для знайдених товарів збільшіть їх вартість на певну суму

In [454]:
print('before:')
print_items_short(
    items.find({'camera': {'$exists': True}}, {'_id': 0, 'model': 1, 'price': 1})
)

items.update_many(
    {'camera': {'$exists': True}},
    {'$inc': {'price': 10000}}
)

print('\nafter:')
print_items_short(
    items.find({'camera': {'$exists': True}}, {'_id': 0, 'model': 1, 'price': 1})
)

before:
{'model': 'iPhone 16 Pro', 'price': 61499}
{'model': 'iPhone 13', 'price': 25499}
{'model': 'Galaxy S25 Ultra', 'price': 67999}
{'model': 'Galaxy A16', 'price': 9499}

after:
{'model': 'iPhone 16 Pro', 'price': 71499}
{'model': 'iPhone 13', 'price': 35499}
{'model': 'Galaxy S25 Ultra', 'price': 77999}
{'model': 'Galaxy A16', 'price': 19499}


# Замовлення (orders)

1. Створіть кілька замовлень з різними наборами товарів, але так щоб один з товарів був у декількох замовленнях

In [455]:
orders = db['orders']

def id_by_model(model):
    return items.find_one({'model': model})['_id']

def price_by_model(model):
    return items.find_one({'model': model})['price']

new_orders = [
    {
        "date" : datetime(2015, 4, 14),
        "total_sum" : sum(price_by_model(m) for m in ['Watch SE']),
        "customer" : {
            "name" : "Andrii",
            "surname" : "Rodinov",
            "phones" : ["9876543", "1234567"],
            "address" : "PTI, Peremohy 37, Kyiv, UA"
        },
        "payment" : {
            "card_owner" : "Andrii Rodinov",
            "cardId" : 12345678
        },
        "items_id" : [id_by_model(m) for m in ['Watch SE']]
    },
    {
        "date" : datetime(2016, 2, 2),
        "total_sum" : sum(price_by_model(m) for m in ['iPhone 13']),
        "customer" : {
            "name" : "Andrii",
            "surname" : "Rodinov",
            "phones" : ["9876543", "1234567"],
            "address" : "PTI, Peremohy 37, Kyiv, UA"
        },
        "payment" : {
            "card_owner" : "Andrii Rodinov",
            "cardId" : 12345678
        },
        "items_id" : [id_by_model(m) for m in ['iPhone 13']]
    },
    {
        "date" : datetime(2016, 3, 3),
        "total_sum" : sum(price_by_model(m) for m in ['Galaxy A16']),
        "customer" : {
            "name" : "Artem",
            "surname" : "Litkovskyi",
            "phones" : ["2345678"],
            "address" : "Vozdvyzhens'ka 55, Kyiv, UA"
        },
        "payment" : {
            "card_owner" : "Artem Litkovskyi",
            "cardId" : 23456789
        },
        "items_id" : [id_by_model(m) for m in ['Galaxy A16']]
    },
    {
        "date" : datetime(2025, 4, 8),
        "total_sum" : sum(price_by_model(m) for m in ['Watch SE', 'UE43T5300AUXUA']),
        "customer" : {
            "name" : "Artem",
            "surname" : "Litkovskyi",
            "phones" : ["2345678"],
            "address" : "Vozdvyzhens'ka 55, Kyiv, UA"
        },
        "payment" : {
            "card_owner" : "Artem Litkovskyi",
            "cardId" : 23456789
        },
        "items_id" : [id_by_model(m) for m in ['Watch SE', 'UE43T5300AUXUA']]
    }
]

orders.insert_many(new_orders)

InsertManyResult([ObjectId('67f55a80c9c92197da108821'), ObjectId('67f55a80c9c92197da108822'), ObjectId('67f55a80c9c92197da108823'), ObjectId('67f55a80c9c92197da108824')], acknowledged=True)

2. Виведіть всі замовлення

In [456]:
print_items_short(orders.find())

{'_id': ObjectId('67f55a80c9c92197da108821'), 'date': datetime.datetime(2015, 4, 14, 0, 0), 'total_sum': 12599, 'customer': {'name': 'Andrii', 'surname': 'Rodinov', 'phones': ['9876543', '1234567'], 'address': 'PTI, Peremohy 37, Kyiv, UA'}, 'payment': {'card_owner': 'Andrii Rodinov', 'cardId': 12345678}, 'items_id': [ObjectId('67f55a80c9c92197da10881b')]}
{'_id': ObjectId('67f55a80c9c92197da108822'), 'date': datetime.datetime(2016, 2, 2, 0, 0), 'total_sum': 35499, 'customer': {'name': 'Andrii', 'surname': 'Rodinov', 'phones': ['9876543', '1234567'], 'address': 'PTI, Peremohy 37, Kyiv, UA'}, 'payment': {'card_owner': 'Andrii Rodinov', 'cardId': 12345678}, 'items_id': [ObjectId('67f55a80c9c92197da108818')]}
{'_id': ObjectId('67f55a80c9c92197da108823'), 'date': datetime.datetime(2016, 3, 3, 0, 0), 'total_sum': 19499, 'customer': {'name': 'Artem', 'surname': 'Litkovskyi', 'phones': ['2345678'], 'address': "Vozdvyzhens'ka 55, Kyiv, UA"}, 'payment': {'card_owner': 'Artem Litkovskyi', 'cardId

3. Виведіть замовлення з вартістю більше певного значення

In [457]:
res = orders.find({'total_sum': {'$gt': 30000}})

print_items_pretty(res)

{
	_id: 67f55a80c9c92197da108822,
	date: 2016-02-02 00:00:00,
	total_sum: 35499,
	customer: {'name': 'Andrii', 'surname': 'Rodinov', 'phones': ['9876543', '1234567'], 'address': 'PTI, Peremohy 37, Kyiv, UA'},
	payment: {'card_owner': 'Andrii Rodinov', 'cardId': 12345678},
	items_id: [ObjectId('67f55a80c9c92197da108818')]
}


4. Знайдіть замовлення зроблені одним замовником

In [458]:
res = orders.find({'customer.name': 'Andrii'})

print_items_pretty(res)

{
	_id: 67f55a80c9c92197da108821,
	date: 2015-04-14 00:00:00,
	total_sum: 12599,
	customer: {'name': 'Andrii', 'surname': 'Rodinov', 'phones': ['9876543', '1234567'], 'address': 'PTI, Peremohy 37, Kyiv, UA'},
	payment: {'card_owner': 'Andrii Rodinov', 'cardId': 12345678},
	items_id: [ObjectId('67f55a80c9c92197da10881b')]
},
{
	_id: 67f55a80c9c92197da108822,
	date: 2016-02-02 00:00:00,
	total_sum: 35499,
	customer: {'name': 'Andrii', 'surname': 'Rodinov', 'phones': ['9876543', '1234567'], 'address': 'PTI, Peremohy 37, Kyiv, UA'},
	payment: {'card_owner': 'Andrii Rodinov', 'cardId': 12345678},
	items_id: [ObjectId('67f55a80c9c92197da108818')]
}


5. Знайдіть всі замовлення з певним товаром (товарами) (шукати можна по ObjectId)

In [459]:
res = orders.find({'items_id': id_by_model('Watch SE')})

print_items_pretty(res)

{
	_id: 67f55a80c9c92197da108821,
	date: 2015-04-14 00:00:00,
	total_sum: 12599,
	customer: {'name': 'Andrii', 'surname': 'Rodinov', 'phones': ['9876543', '1234567'], 'address': 'PTI, Peremohy 37, Kyiv, UA'},
	payment: {'card_owner': 'Andrii Rodinov', 'cardId': 12345678},
	items_id: [ObjectId('67f55a80c9c92197da10881b')]
},
{
	_id: 67f55a80c9c92197da108824,
	date: 2025-04-08 00:00:00,
	total_sum: 27798,
	customer: {'name': 'Artem', 'surname': 'Litkovskyi', 'phones': ['2345678'], 'address': "Vozdvyzhens'ka 55, Kyiv, UA"},
	payment: {'card_owner': 'Artem Litkovskyi', 'cardId': 23456789},
	items_id: [ObjectId('67f55a80c9c92197da10881b'), ObjectId('67f55a80c9c92197da10881e')]
}


6. Додайте в усі замовлення з певним товаром ще один товар і збільшіть існуючу вартість замовлення на деяке значення Х

In [460]:
print('before:')
print_items_short(
    orders.find({'items_id': id_by_model('Watch SE')}, {'_id': 0, 'total_sum': 1, 'items_id': 1})
)

orders.update_many(
    {'items_id': id_by_model('Watch SE')},
    {
        '$push': {'items_id': id_by_model('50UR78006LK')},
        '$inc': {'total_sum': price_by_model('50UR78006LK')}
    }
)

print('\nafter:')
print_items_short(
    orders.find({'items_id': id_by_model('Watch SE')}, {'_id': 0, 'total_sum': 1, 'items_id': 1})
)

before:
{'total_sum': 12599, 'items_id': [ObjectId('67f55a80c9c92197da10881b')]}
{'total_sum': 27798, 'items_id': [ObjectId('67f55a80c9c92197da10881b'), ObjectId('67f55a80c9c92197da10881e')]}

after:
{'total_sum': 34098, 'items_id': [ObjectId('67f55a80c9c92197da10881b'), ObjectId('67f55a80c9c92197da108820')]}
{'total_sum': 49297, 'items_id': [ObjectId('67f55a80c9c92197da10881b'), ObjectId('67f55a80c9c92197da10881e'), ObjectId('67f55a80c9c92197da108820')]}


7. Виведіть кількість товарів в певному замовленні

In [461]:
print(len(orders.find_one()['items_id']))

2


8. Виведіть тільки інформацію про кастомера і номери кредитної карт, для замовлень вартість яких перевищує певну суму

In [462]:
res = orders.find({'total_sum': {'$gt': 30000}}, {'_id': 0, 'customer': 1, 'payment.cardId': 1})

print_items_pretty(res)

{
	customer: {'name': 'Andrii', 'surname': 'Rodinov', 'phones': ['9876543', '1234567'], 'address': 'PTI, Peremohy 37, Kyiv, UA'},
	payment: {'cardId': 12345678}
},
{
	customer: {'name': 'Andrii', 'surname': 'Rodinov', 'phones': ['9876543', '1234567'], 'address': 'PTI, Peremohy 37, Kyiv, UA'},
	payment: {'cardId': 12345678}
},
{
	customer: {'name': 'Artem', 'surname': 'Litkovskyi', 'phones': ['2345678'], 'address': "Vozdvyzhens'ka 55, Kyiv, UA"},
	payment: {'cardId': 23456789}
}


9. Видаліть товар з замовлень, зроблених за певний період дат

In [463]:
print('before:')
print_items_short(
    orders.find(
        {'date': {'$gt': datetime(2016, 1, 1), '$lt': datetime(2016, 12, 31)}},
        {'_id': 0, 'date': 1, 'items_id': 1}
    )
)

orders.update_many(
    {'date': {'$gt': datetime(2016, 1, 1), '$lt': datetime(2016, 12, 31)}},
    {'$pull': {'items_id': id_by_model('iPhone 13')}}
)

print('\nafter:')
print_items_short(
    orders.find(
        {'date': {'$gt': datetime(2016, 1, 1), '$lt': datetime(2016, 12, 31)}},
        {'_id': 0, 'date': 1, 'items_id': 1}
    )
)

before:
{'date': datetime.datetime(2016, 2, 2, 0, 0), 'items_id': [ObjectId('67f55a80c9c92197da108818')]}
{'date': datetime.datetime(2016, 3, 3, 0, 0), 'items_id': [ObjectId('67f55a80c9c92197da10881a')]}

after:
{'date': datetime.datetime(2016, 2, 2, 0, 0), 'items_id': []}
{'date': datetime.datetime(2016, 3, 3, 0, 0), 'items_id': [ObjectId('67f55a80c9c92197da10881a')]}


10. Перейменуйте у всіх замовлення ім'я (прізвище) замовника

In [464]:
orders.update_many(
    {},
    {'$set': {'customer.surname': 'Brooklyn'}}
)

print_items_short(
    orders.find(
        {},
        {'_id': 0, 'customer': 1}
    )
)

{'customer': {'name': 'Andrii', 'surname': 'Brooklyn', 'phones': ['9876543', '1234567'], 'address': 'PTI, Peremohy 37, Kyiv, UA'}}
{'customer': {'name': 'Andrii', 'surname': 'Brooklyn', 'phones': ['9876543', '1234567'], 'address': 'PTI, Peremohy 37, Kyiv, UA'}}
{'customer': {'name': 'Artem', 'surname': 'Brooklyn', 'phones': ['2345678'], 'address': "Vozdvyzhens'ka 55, Kyiv, UA"}}
{'customer': {'name': 'Artem', 'surname': 'Brooklyn', 'phones': ['2345678'], 'address': "Vozdvyzhens'ka 55, Kyiv, UA"}}


11. Знайдіть замовлення зроблені одним замовником, і виведіть тільки інформацію про кастомера та товари у замовлені підставивши замість ObjectId("***") назви товарів та їх вартість (аналог join-а між таблицями orders та items).

In [465]:
res = orders.aggregate([
    {
        '$match': {'customer.name': 'Artem'}
    },
    {
        '$lookup': {
            'from': 'items',
            'localField': 'items_id',
            'foreignField': '_id',
            'as': 'items_obj'
        }
    },
    {
        '$project': {'_id': 0, 'customer': 1, 'items_obj': 1}
    }
])

print_items_pretty(res, very_pretty=True)

{
	customer: {
		name: Artem,
		surname: Brooklyn,
		phones: ['2345678'],
		address: Vozdvyzhens'ka 55, Kyiv, UA
	},
	items_obj: [
		{
			_id: 67f55a80c9c92197da10881a,
			category: Phone,
			model: Galaxy A16,
			producer: Samsung,
			price: 19499,
			display: 6.7,
			camera: [50, 5, 2],
			os: Android
		}
	]
},
{
	customer: {
		name: Artem,
		surname: Brooklyn,
		phones: ['2345678'],
		address: Vozdvyzhens'ka 55, Kyiv, UA
	},
	items_obj: [
		{
			_id: 67f55a80c9c92197da108820,
			category: TV,
			model: 50UR78006LK,
			producer: LG,
			price: 21499,
			display: 50,
			os: WebOS,
			refresh_rate: 60
		},
		{
			_id: 67f55a80c9c92197da10881e,
			category: TV,
			model: UE43T5300AUXUA,
			producer: Samsung,
			price: 15199,
			display: 43,
			os: Tizen,
			refresh_rate: 60
		},
		{
			_id: 67f55a80c9c92197da10881b,
			category: Smart Watch,
			model: Watch SE,
			producer: Apple,
			price: 12599,
			display: 40,
			wrist: [130, 180],
			os: WatchOS
		}
	]
}


# Сapped collection

Створіть Сapped collection яка б містила 5 останніх відгуків на наш інтернет-магазин. Структуру запису визначити самостійно.

In [466]:
db.create_collection('recent_reviews', capped=True, size=800, max=5)

recent_reviews = db['recent_reviews']

new_reviews = [
    {
        'date': datetime(2025, 1, 1),
        'author': 'Artem',
        'rating': 5,
        'text': 'I highly recommend this business.' 
    },
    {
        'date': datetime(2025, 1, 3),
        'author': 'Andrii',
        'rating': 5,
        'text': 'I was completely impressed with their professionalism and customer service.' 
    },
    {
        'date': datetime(2025, 2, 12),
        'author': 'Giselle',
        'rating': 4,
        'text': 'The ambiance here is always inviting and comfortable.' 
    },
    {
        'date': datetime(2025, 3, 25),
        'author': 'Elise',
        'rating': 4,
        'text': 'Efficiency and punctuality are hallmarks of their service.' 
    },
    {
        'date': datetime(2025, 4, 1),
        'author': 'Grigory',
        'rating': 4,
        'text': 'They are reliable, honest and operate with integrity.' 
    }
]

recent_reviews.insert_many(new_reviews)

print_items_short(recent_reviews.find())

{'_id': ObjectId('67f55a80c9c92197da108825'), 'date': datetime.datetime(2025, 1, 1, 0, 0), 'author': 'Artem', 'rating': 5, 'text': 'I highly recommend this business.'}
{'_id': ObjectId('67f55a80c9c92197da108826'), 'date': datetime.datetime(2025, 1, 3, 0, 0), 'author': 'Andrii', 'rating': 5, 'text': 'I was completely impressed with their professionalism and customer service.'}
{'_id': ObjectId('67f55a80c9c92197da108827'), 'date': datetime.datetime(2025, 2, 12, 0, 0), 'author': 'Giselle', 'rating': 4, 'text': 'The ambiance here is always inviting and comfortable.'}
{'_id': ObjectId('67f55a80c9c92197da108828'), 'date': datetime.datetime(2025, 3, 25, 0, 0), 'author': 'Elise', 'rating': 4, 'text': 'Efficiency and punctuality are hallmarks of their service.'}
{'_id': ObjectId('67f55a80c9c92197da108829'), 'date': datetime.datetime(2025, 4, 1, 0, 0), 'author': 'Grigory', 'rating': 4, 'text': 'They are reliable, honest and operate with integrity.'}


1. Перевірте що при досягненні обмеження старі відгуки будуть затиратись.

In [467]:
new_reviews = [
    {
        'date': datetime(2025, 4, 7),
        'author': 'Vitalii',
        'rating': 5,
        'text': 'My brother recommended this store to me. Their service is even better than I expected.' 
    },
    {
        'date': datetime(2025, 4, 8),
        'author': 'Beatrice',
        'rating': 1,
        'text': '.' 
    }
]

recent_reviews.insert_many(new_reviews)

print_items_short(recent_reviews.find())

{'_id': ObjectId('67f55a80c9c92197da108827'), 'date': datetime.datetime(2025, 2, 12, 0, 0), 'author': 'Giselle', 'rating': 4, 'text': 'The ambiance here is always inviting and comfortable.'}
{'_id': ObjectId('67f55a80c9c92197da108828'), 'date': datetime.datetime(2025, 3, 25, 0, 0), 'author': 'Elise', 'rating': 4, 'text': 'Efficiency and punctuality are hallmarks of their service.'}
{'_id': ObjectId('67f55a80c9c92197da108829'), 'date': datetime.datetime(2025, 4, 1, 0, 0), 'author': 'Grigory', 'rating': 4, 'text': 'They are reliable, honest and operate with integrity.'}
{'_id': ObjectId('67f55a80c9c92197da10882a'), 'date': datetime.datetime(2025, 4, 7, 0, 0), 'author': 'Vitalii', 'rating': 5, 'text': 'My brother recommended this store to me. Their service is even better than I expected.'}
{'_id': ObjectId('67f55a80c9c92197da10882b'), 'date': datetime.datetime(2025, 4, 8, 0, 0), 'author': 'Beatrice', 'rating': 1, 'text': '.'}
